In [18]:
import os
import torch
import torchaudio
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import WhisperFeatureExtractor
import pandas as pd

In [19]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda is available")
else:
    device = torch.device("cpu")

cuda is available


In [20]:
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

In [21]:

#audio dataset root directory
dataset_path = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios"

In [22]:
dir_lis = os.listdir(dataset_path)
path_list = []
gender_list = []
emotion_list=[]

In [23]:
emotion_dic = {
    '01' : 'happy',
    '02' : 'neutral',
    '03' : 'sad',
    '04' : 'angry',
    '05' : 'fear',
    '06' : 'disgust',
    '07' : 'surprise',
    '08' : 'calm',
}

In [24]:
intensity = {
    '01' : 'high',
    '02' : 'low',
}

In [36]:
def process_files(directory):
    for root, dirs, files in os.walk(directory):
        for audio_file in files:
            part = audio_file.split('.')[0]
            part_split = part.split('-')
            if len(part_split) >= 7:
                key = part_split[2]
                if key in emotion_dic:
                    gender_code = int(part_split[6])
                    path_list.append(os.path.join(root, audio_file))
                    gender_list.append('female' if gender_code & 1 == 0 else 'male')
                    emotion_list.append(emotion_dic[key])

# Process files in each directory
for directory in dir_lis:
    directory_path = os.path.join(dataset_path, directory)
    if os.path.isdir(directory_path):
        process_files(directory_path)

# the number of files processed
num_files_processed = len(path_list)
print("Number of files processed:", num_files_processed)

        
        
# Create DataFrame
ravdess_df = pd.DataFrame({
    'path': path_list,
    'sex': gender_list,
    'emotion': emotion_list
})

ravdess_df.head()

Number of files processed: 2881


,path,sex,emotion
0,/mnt/c/users/admin/desktop/F21RP/datasets/ravd...,male,happy
1,/mnt/c/users/admin/desktop/F21RP/datasets/ravd...,male,happy
2,/mnt/c/users/admin/desktop/F21RP/datasets/ravd...,male,happy
3,/mnt/c/users/admin/desktop/F21RP/datasets/ravd...,male,happy
4,/mnt/c/users/admin/desktop/F21RP/datasets/ravd...,male,happy


In [34]:
import torch
import torchaudio
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset
import os
import pandas as pd
import numpy as np

# Initializing the WhisperModel
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

# directory containing the audio files
directory = "/mnt/c/users/admin/desktop/F21RP/datasets/ravdess/audios"

# list to store the extracted features
features_list = []

#function for loading the audio file
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    return waveform

# Define a function to process files and extract features
def process_files(directory):
    for root, dirs, files in os.walk(directory):
        for audio_file in files:
            part = audio_file.split('.')[0]
            part_split = part.split('-')
            if len(part_split) >= 7:
                key = part_split[2]
                if key in emotion_dic:
                    gender_code = int(part_split[6])
                    audio_path = os.path.join(root, audio_file)
                    
                    # Loading the audio file
                    audio = load_audio(audio_path)
                    
                    # Extract features using the feature extractor
                    inputs = feature_extractor(audio, return_tensors="pt")
                    input_features = inputs.input_features
                    decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
                    last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).last_hidden_state
                    
                    # Check the shape of the extracted features
                    features_shape = last_hidden_state.shape[1:]
                    
                    #padding if necessary
                    if features_shape != (3, 2):
                        required_shape = (3, 2)  # Adjust the shape to meet requirement
                        difference = (np.array(required_shape) - np.array(features_shape)).tolist()
                        last_hidden_state = np.pad(last_hidden_state, ((0, 0), (0, difference[0]), (0, difference[1])), mode='constant')
                    
                    # Adding the features and metadata to the list
                    features_list.append({
                        'path': audio_path,
                        'sex': 'female' if gender_code & 1 == 0 else 'male',
                        'emotion': emotion_dic[key],
                        'features': last_hidden_state.squeeze()  # Removing unnecessary dimensions
                    })

# Process files in the directory
process_files(directory)

#DataFrame from the features list
ravdess_df = pd.DataFrame(features_list)

# Save the DataFrame to a CSV file
ravdess_df.to_csv("ravdess_features.csv", index=False)


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (3,2)